In [ ]:
import pandas as pd
import re
from bs4 import BeautifulSoup

In [ ]:
def clean_html_text(html_text):
    """
    Limpia texto quwe contiene etiquetas HTML manteniendo saltos de línea <br> y enlaces.
    Args:
        html_text (str): Texto con etiquetas HTML
    Returns:
        str: Texto limpio con saltos de línea y enlaces preservados
    """
    # Verificar si el valor es una cadena
    if not isinstance(html_text, str):
        return html_text

    # Paso 1: Usar BeautifulSoup para manejar el HTML
    soup = BeautifulSoup(html_text, "html.parser")

    # Paso 2: Reemplazar <br> tags con un marcador especial
    for br in soup.find_all("br"):
        br.replace_with("\n\n")

    # Paso 3: Procesar los enlaces de diversas etiquetas
    tags_to_process = ["a", "link", "iframe", "script", "img", "audio", "video", "area"]
    for tag_name in tags_to_process:
        for tag in soup.find_all(tag_name):
            # Obtener todos los atributos que podrían contener URLs
            attributes = tag.attrs
            for attr, value in attributes.items():
                if isinstance(value, str) and (
                    "http://" in value or "https://" in value
                ):
                    # Extraer y formatear los links
                    try:
                        if tag_name == "a":
                            href = tag.get("href", "")
                            text = tag.get_text()
                            if href and text:
                                tag.replace_with(
                                    f" ({href})"
                                )  # Mantener el texto y agregar el link

                        elif tag_name in (
                            "img",
                            "script",
                            "audio",
                            "video",
                            "iframe",
                            "area",
                        ):
                            if "src" in tag.attrs:
                                tag.replace_with(f" ({tag['src']})")

                        elif tag_name == "link":
                            if "href" in tag.attrs:
                                tag.replace_with(f" ({tag['href']})")
                    except:
                        tag.replace_with(f" ({value})")

    # Paso 4: Obtener el texto
    text = soup.get_text()

    # Paso 5: Limpiar caracteres especiales y espacios múltiples (opcional, mantenido de tu código original)
    # clean_text = re.sub(r"\s+", " ", text)  # Reemplazar múltiples espacios con uno solo
    # clean_text = re.sub(r"&nbsp;", " ", clean_text)  # Reemplazar &nbsp;

    # Paso 6: Limpiar espacios al inicio y final manteniendo los saltos de línea internos
    lines = text.split("\n")
    clean_lines = [line.strip() for line in lines]
    clean_text = "\n\n".join(filter(None, clean_lines))

    return clean_text

# Leer el archivo CSV
df = df_fixed.copy()

# Mostrar el número total de registros
print("Total de registros antes de la limpieza:", len(df))

# Aplicar la función de limpieza a todas las columnas que contienen texto
# columnas_texto = df.select_dtypes(include=["object"]).columns
# for columna in columnas_texto:
#     df[columna] = df[columna].apply(clean_html_text)

df['texto'] = df['texto'].apply(clean_html_text)

print("\nTotal de registros después de la limpieza:", len(df))

# Mostrar las primeras 3 filas del DataFrame limpio
# print("\nPrimeras 3 filas del DataFrame limpio:")
df.head(15)